In [39]:
import utils

import re
from collections import Counter
from dataclasses import dataclass
import numpy as np
from collections import deque

## Day 10: Hoof It

[#](https://adventofcode.com/2024/day/10) Another map problem! This time with a 3d map!

Make a 2d grid from the puzzle input, where the numbers `0-9` represent the height at each position.

Find all the good hiking trails on the map. Good trails are defined by being as long as possible and having an even uphill slope. Trails should:
1. Start from 0, end at 9
2. Increase by 1 at each step
3. Only move up, down, left or right

Every trailhead is scocred by the number of height 9 positions you can reach from there on a hiking trail.

In [58]:
sample_input: str = """
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
"""

puzzle_input = utils.get_input(10, splitlines=False)

In [59]:
def parse_input(input_str=sample_input, debug=False):
    """returns 2d grid"""
    return [[int(char) for char in row] for row in input_str.strip().splitlines()]


grid = parse_input(sample_input, True)
print(grid)

[[8, 9, 0, 1, 0, 1, 2, 3], [7, 8, 1, 2, 1, 8, 7, 4], [8, 7, 4, 3, 0, 9, 6, 5], [9, 6, 5, 4, 9, 8, 7, 4], [4, 5, 6, 7, 8, 9, 0, 3], [3, 2, 0, 1, 9, 0, 1, 2], [0, 1, 3, 2, 9, 8, 0, 1], [1, 0, 4, 5, 6, 7, 3, 2]]


In [60]:
def find_trailheads(grid=grid):
    """returns trailhead positions"""
    trailheads = []
    for y, row in enumerate(grid):
        for x, height in enumerate(row):
            if height == 0:
                trailheads.append((x, y))
    return trailheads


trailheads = find_trailheads(grid)
trailheads

[(2, 0), (4, 0), (4, 2), (6, 4), (2, 5), (5, 5), (0, 6), (6, 6), (1, 7)]

Bread first search: from a given starting node:
1. check unvisited neighbours, add to queue if valid , and mark as visitied
2. now keep doing this for all nodes in the queue, and keep adding valid nodes
3. Stop when target found, or increment a counter if finding multiple targets.

BFS key point: track visited nodes, so you don't re-visit them, else can cause infinite loops and return longer than optimal paths



In [ ]:
def bfs(pos, grid=grid):
    """bread first search, returns the number of 9 positions reachable from a given pos"""
    x, y = pos
    X = len(grid[0])
    Y = len(grid)

    visited = set() # can't reuse positions in a path
    count_9 = 0
    queue = deque([pos])

    while queue:
        (x, y) = queue.popleft()
        if (x, y) in visited:
            continue
        visited.add((x, y))

        if grid[y][x] == 9:
            count_9 += 1

        # check in 4 directions only
        for dx, dy in [(0, -1), (0, 1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy

            if 0 <= nx < X and 0 <= ny < Y and grid[ny][nx] == grid[y][x] + 1:
                queue.append((nx, ny))

    return count_9


bfs((2, 0))

5

In [88]:
def search(pos, grid=grid, bfs=True, track_visited=True, debug=False):
    """bread first search, returns the number of 9 positions reachable from a given pos"""
    x, y = pos
    X = len(grid[0])
    Y = len(grid)

    visited = set()  # can't reuse positions in a path
    count_9 = 0
    queue = deque([pos])

    while queue:

        if bfs:
            (x, y) = queue.popleft()  # bfs
        else:
            (x, y) = queue.pop()  # dfs

        if track_visited:
            if (x, y) in visited:
                continue
            visited.add((x, y))

        if grid[y][x] == 9:
            count_9 += 1

        # check in 4 directions only
        for dx, dy in [(0, -1), (0, 1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy

            if 0 <= nx < X and 0 <= ny < Y and grid[ny][nx] == grid[y][x] + 1:
                queue.append((nx, ny))

    return count_9


search((2, 0))

5

In [89]:
def solve(inp: str = sample_input, bfs=True, track_visited=True, debug: bool = False):
    grid = parse_input(inp)
    trailheads = find_trailheads(grid)
    trail_scores = []
    for head in trailheads:
        score = search(head, grid, bfs, track_visited, debug)
        trail_scores.append(score)

    if debug:
        print(f"{score} {trail_scores}")

    return {"result": sum(trail_scores), "trail_scores": trail_scores}


assert solve(sample_input, bfs=True, debug=True)["result"] == 36  # sample ans check

results = solve(puzzle_input, debug=False)
print(f"Part 1: {results["result"]}")

5 [5, 6, 5, 3, 1, 3, 5, 3, 5]
Part 1: 517


## Part 2

Each trailhead now gets a rating - which is the number of distinct hiking trails which begin at that trailhead.

This is really a english problem - part 1 you can't revisit points, while in part 2 you can. This was the fastest part 2 of all the days so far!

Thanks to using BFS or DFS above - the only change I had to do was remove tracking visited positions.



In [93]:
assert solve(track_visited=False)["result"] == 81
solve(puzzle_input, track_visited=False)["result"]

1116

In [95]:
assert solve(bfs=False, track_visited=False)["result"] == 81
solve(puzzle_input, bfs=False, track_visited=False)["result"]

1116